In [1]:
import h5py
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation, ArtistAnimation, PillowWriter
import pandas as pd
from pandas import HDFStore,DataFrame

## Object Quantification
- Example Notebook of how you would quantify segmentation images. 

In [33]:
#Path to exported features as a csv, can also export as hdf5. 
path722 = "/Volumes/comp_micro/Projects/nucleolus/labelfree_segmentation/722_features-0_table.csv"
nuc_722 = pd.read_csv(path722)

In [34]:
#Only keeping Label 1 objects corresponding to the segmented nucleolus
nuc_722_f = nuc_722[nuc_722["Predicted Class"]=="Label 1"]

In [35]:
density = nuc_722_f["Mean Intensity_0"]
density_n = nuc_722_f["Mean Intensity in neighborhood_0"]

In [36]:
#Rescale intensity values to physical optical path length values [0.5,2]
nuc_722_f["Mean Intensity_0"] = np.interp(density, (density.min(), density.max()), (-0.5, +2))
nuc_722_f["Mean Intensity in neighborhood_0"] = np.interp(density_n, (density_n.min(), density_n.max()), (-0.5, +2))

/Users/andrew.chen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/andrew.chen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [37]:
#Calculate Aspect Ratio (Width/Height)
nuc_722_f["Aspect Ratio"] = nuc_722_f["Radii of the object_0"]/nuc_722_f["Radii of the object_1"]

/Users/andrew.chen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
#Only keeping columns of interest
nuc_722_f = nuc_722_f[['timestep',  'Size in pixels', 'Aspect Ratio',
                       'Mean Intensity_0', 'Mean Intensity in neighborhood_0']]

In [39]:
#Converting size to microns
nuc_722_f["Size in pixels"] = nuc_722_f["Size in pixels"]*(6.45/63)

In [40]:
nuc_722_f.head()

,timestep,Size in pixels,Aspect Ratio,Mean Intensity_0,Mean Intensity in neighborhood_0
2,0,30.100000,3.688112,0.325997,0.147045
3,0,27.130952,7.698977,0.939571,0.149617
4,0,49.654762,3.152703,0.732901,0.132151
6,0,23.035714,1.046104,0.554148,0.015202
7,0,30.919048,1.710274,-0.234431,-0.045874


In [41]:
nuc_722_f_g = nuc_722_f.groupby("timestep")

In [43]:
#Extracting data from pandas dataframe to a dictionary.
datamatrix = {}
for time in np.arange(60):
    data = nuc_722_f_g.get_group(time).to_numpy()[:,1:]
    datamatrix[time] = data

In [44]:
#Automatically finding good bins. 
_, bins0 = np.histogram(datamatrix[0][:,0])
_, bins1 = np.histogram(datamatrix[0][:,1])
_, bins2 = np.histogram(datamatrix[0][:,2])
_, bins3 = np.histogram(datamatrix[0][:,3])

## Making Animated Histograms of Object measurements over time

In [56]:
plt.rcParams['animation.ffmpeg_path'] = '/usr/local/bin/ffmpeg'

In [57]:
fig, ax = plt.subplots(nrows=2,ncols=2, figsize = (10, 10))
plt.close(fig)

def updateData(i):
    #if curr <=2: return
    ax[0,0].clear()
    ax[0,1].clear()
    ax[1,0].clear()
    ax[1,1].clear()
    
    ax[0,0].set_title("Size")
    ax[0,1].set_title("Aspect Ratio")
    ax[1,0].set_title("Mean Density")
    ax[1,1].set_title("Mean Density in Neighborhood")
    
    ax[0,0].hist(datamatrix[i][:,0], normed = True, bins = bins0, alpha = 0.5)
    ax[0,1].hist(datamatrix[i][:,1], normed = True, bins = bins1, alpha = 0.5)
    ax[1,0].hist(datamatrix[i][:,2], normed = True, bins = bins2, alpha = 0.5)
    ax[1,1].hist(datamatrix[i][:,3], normed = True, bins = bins3, alpha = 0.5)

simulation = FuncAnimation(fig, updateData, interval=200, repeat=False)

In [58]:
name = "Nucleolus_Quantified.mp4"

In [59]:
simulation.save(name, writer = 'ffmpeg')

/Users/andrew.chen/anaconda3/lib/python3.7/site-packages/matplotlib/axes/_axes.py:6521: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  alternative="'density'", removal="3.1")


KeyError: 60